In [238]:
import json
import pprint

from jsonschema.validators import Draft7Validator

"""
Open a Mosdex file and validate it against the schema
"""

MOSDEX_SCHEMA_FILE = "MOSDEXSchemaV2-1.json"
MOSDEX_FILE = "sailco_2-1.json"

with open(MOSDEX_SCHEMA_FILE, "r") as f:
    schema = json.load(f)
validator = Draft7Validator(schema)

with open(MOSDEX_FILE, "r") as f:
    mosdex = json.load(f)

if not validator.is_valid(mosdex):
    print(f"File {MOSDEX_FILE} is not a valid Mosdex file.")
    pp = pprint.PrettyPrinter(indent=4)
    for error in sorted(validator.iter_errors(mosdex), key=str):
        print()
        pp.pprint(error.message)
else:
    print(f"File {MOSDEX_FILE} is a valid instance of schema {MOSDEX_SCHEMA_FILE}.")


File sailco_2-1-stochastic.json is a valid instance of schema MOSDEXSchemaV2-1.json.


In [239]:
# Get reference to a module that has KIND == MODEL
model = {}
for module in mosdex['MODULES']:
    if module['KIND'] == 'MODEL':
        model = module
        break

print(f"Got handle to MODEL: {model['NAME']}")
print(f"The sections of the model are {list(model.keys())}")
print(f"\tNAME: {model['NAME']}")
print(f"\tCLASS: {model['CLASS']}")
print(f"\tKIND: {model['KIND']}")
print(f"\tTABLES: there are {len(model['TABLES'])} tables:")
for table in model['TABLES']:
    print(f"\t\t{table['NAME']:10s} \t class/kind: {table['CLASS']}/{table['KIND']}")

Got handle to MODEL: sailco
The sections of the model are ['NAME', 'CLASS', 'KIND', 'HEADING', 'TABLES']
	NAME: sailco
	CLASS: MODULE
	KIND: MODEL
	TABLES: there are 11 tables:
		states     	 class/kind: TREE/INPUT
		demands    	 class/kind: DATA/INPUT
		parameters 	 class/kind: DATA/INPUT
		regular    	 class/kind: VARIABLE/CONTINUOUS
		extra      	 class/kind: VARIABLE/CONTINUOUS
		inventory  	 class/kind: VARIABLE/CONTINUOUS
		cost       	 class/kind: VARIABLE/CONTINUOUS
		ctInventory 	 class/kind: CONSTRAINT/LINEAR
		ctInventoryMatrix 	 class/kind: MATRIX/LINEAR
		ctCost     	 class/kind: CONSTRAINT/LINEAR
		ctCostMatrix 	 class/kind: MATRIX/LINEAR


In [240]:
"""
Initialize the database engine
"""
from sqlalchemy import create_engine, Integer, String, Double, ForeignKey

engine = create_engine('sqlite:///:memory:')

from sqlalchemy.orm import declarative_base, mapped_column

Base = declarative_base()
Base.metadata.drop_all(bind=engine)

# Initialize the Stages, Constraints, Variables, and Matrix tables
db_tables = {}
for key in ['variables', 'constraints', 'matrix']:
    db_tables[key] = {}
    db_tables[key]['table_instance_name'] = key
    db_tables[key]['table_class_name'] = model['NAME'].capitalize() + key.capitalize()
    table_attr = { '__tablename__': db_tables[key]['table_instance_name'] ,
                   'id': mapped_column(Integer, primary_key=True),
                   'state': mapped_column(String, ),
                   }
    db_tables[key]['table_class'] = type(db_tables[key]['table_class_name'], (Base,), table_attr)

# Initialize each model table
for table_ in model['TABLES']:
    
    # record table metadata in db_tables
    key = table_['NAME']
    db_tables[key] = {}
    db_tables[key]['table_instance_name'] = key
    db_tables[key]['table_class_name'] = model['NAME'].capitalize() + key.capitalize()
    
    # mosdex metadata
    db_tables[key]['mosdex_json'] = table_
    db_tables[key]['mosdex_schema'] = table_['SCHEMA']
    db_tables[key]['mosdex_class'] = table_['CLASS']
    db_tables[key]['mosdex_kind'] = table_['KIND']
    
       
    # local variables
    table_name = db_tables[key]['table_instance_name']
    table_schema = db_tables[key]['mosdex_schema']
    
    # INPUTS don't have stage columns
    if db_tables[key]['mosdex_kind'] == 'INPUT':
        table_attr = { '__tablename__': table_name ,
                   'id': mapped_column(Integer, primary_key=True),
                   }
    # VARIABLE, CONSTRAINT, and MATRIX do have stage columns
    if db_tables[key]['mosdex_class'] in ['VARIABLE', 'CONSTRAINT', 'MATRIX']:
        table_attr = { '__tablename__': table_name ,
                   'id': mapped_column(Integer, primary_key=True), 
                   'state': mapped_column(String, ),
                }
          
    # Apply the SCHEMA
    for name, kind in zip(table_schema['NAME'], table_schema['KIND']):
        
        # Column type
        if kind == 'INTEGER':
            type_col = Integer
        elif kind == 'DOUBLE' or kind == 'DOUBLE_FUNCTION':
            type_col = Double
        elif kind == 'STRING':
            type_col = String
        else:
            print(f"Error, type {kind} is not supported.  Detected in Table {table_name}")
            break
        
        # Relationships
        # - primary_key flag 
        # - Foreign Key relationship 
        if 'KEYS' in table_schema and name in table_schema['KEYS']:
            # Primary Key
            table_attr[name] = mapped_column(type_col, primary_key=True)
        if 'FOREIGN_KEYS' in table_schema and name in table_schema['FOREIGN_KEYS']:
            # Foreign Key
            f_key = table_schema['FOREIGN_KEYS'][name]
            table_attr[name] = mapped_column(type_col, ForeignKey(f_key))
        else:
            table_attr[name] = mapped_column(type_col)
    
    # Declarative instantiation of the table     
    db_tables[key]['table_class'] = type(db_tables[key]['table_class_name'], (Base,), table_attr)
    
    
Base.metadata.create_all(engine,  )
        
print(f"Database tables created")
for table in Base.metadata.tables.keys():
    print(f"\t{table}")
    print(f"\t\t{Base.metadata.tables[table].columns.keys()}")
    

Database tables created
	variables
		['id', 'state']
	constraints
		['id', 'state']
	matrix
		['id', 'state']
	states
		['id', 'state', 'ancestor', 'period']
	demands
		['id', 'period', 'demand']
	parameters
		['id', 'regularCost', 'extraCost', 'capacity', 'initialInventory', 'inventoryCost']
	regular
		['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	extra
		['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	inventory
		['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	cost
		['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	ctInventory
		['id', 'state', 'rowName', 'sense', 'RHS', 'dualValue']
	ctInventoryMatrix
		['id', 'state', 'rowName', 'regularName', 'regularCoefficient', 'extraName', 'extraCoefficient', 'inventoryName', 'inventoryCoefficient', 'laggedInventoryName', 'laggedInventoryCoefficient']
	ctCost
		['id', 'state', 'rowName', 'sense', 'RHS', 'dualValue']
	ctCostMatrix
		['id', 'state', 'row

In [241]:
from sqlalchemy import text
from sqlalchemy.orm import Session

"""
Load the INSTANCES
"""
import pandas as pd
import numpy as np

for key in db_tables.keys():
    # Not a mosdex object
    if 'mosdex_json' not in db_tables[key]:
        continue
    
    # Load INSTANCE     
    if "INSTANCE" in db_tables[key]['mosdex_json']:
        # Get column names 
        col_names = db_tables[key]['mosdex_schema']['NAME']
        
        # Create a dataframe from the INSTANCE arrays
        data_df = pd.DataFrame(np.vstack(db_tables[key]['mosdex_json']['INSTANCE']), columns=col_names)
        print("\n>>DATAFRAME<< ")
        print(data_df.head())

        # Push the dataframe to the table
        with Session(engine) as session, session.begin():
            data_df.to_sql(name=db_tables[key]['table_instance_name'], con=session.connection(),
                                   if_exists='append', index=False)
            session.flush()
            stmt = "select * from " + db_tables[key]['table_instance_name']
            rows = session.execute(text(stmt))
            
            print("\n>>TABLE<<")
            for row in rows:
                print(row)
    else:
        # Load in another way
        pass
    




>>DATAFRAME<< 
   state  ancestor  period
0      1         0       1
1      2         1       2
2      3         2       3
3      4         3       4

>>TABLE<<
(1, 1, 0, 1)
(2, 2, 1, 2)
(3, 3, 2, 3)
(4, 4, 3, 4)

>>DATAFRAME<< 
   period  demand
0     1.0    40.0
1     2.0    60.0
2     3.0    75.0
3     4.0    25.0

>>TABLE<<
(1, 1, 40.0)
(2, 2, 60.0)
(3, 3, 75.0)
(4, 4, 25.0)

>>DATAFRAME<< 
   regularCost  extraCost  capacity  initialInventory  inventoryCost
0        400.0      450.0      40.0              10.0           20.0

>>TABLE<<
(1, 400.0, 450.0, 40.0, 10.0, 20.0)


In [242]:
"""
Process VARIABLES
"""

for key in db_tables.keys():
    if 'mosdex_json' not in db_tables[key]:
        continue        
    if "VARIABLE" == db_tables[key]['mosdex_json']['CLASS']:
        for statement in db_tables[key]['mosdex_json']['QUERY']:
            
            insert_array = db_tables[key]['mosdex_schema']['NAME']
            select_array = statement['SELECT']
            from_array = statement['FROM']
            
            insert_stmt = "INSERT INTO " + key + '(' + ",".join(insert_array) + ')'
            select_stmt = "SELECT " + ",".join(select_array)
            from_stmt = "FROM " + ",".join(from_array)
            stmt = insert_stmt + ' ' + select_stmt + " " + from_stmt
            # print(stmt)
            with Session(engine) as session, session.begin():
                session.execute(text(stmt))
                
        with Session(engine) as session:
            stmt = "select * from " + db_tables[key]['table_instance_name']
            rows = session.execute(text(stmt)).fetchall()
            print(f"\n>>TABLE<< {db_tables[key]['table_instance_name']}")
            print(f"{Base.metadata.tables[key].columns.keys()}")
            for row in rows:
                print(row)





>>TABLE<< regular
['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
(1, 1, 'regular_1', 0.0, 40.0, None)
(2, 2, 'regular_2', 0.0, 40.0, None)
(3, 3, 'regular_3', 0.0, 40.0, None)
(4, 4, 'regular_4', 0.0, 40.0, None)

>>TABLE<< extra
['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
(1, 1, 'extra_1', 0.0, 'Infinity', None)
(2, 2, 'extra_2', 0.0, 'Infinity', None)
(3, 3, 'extra_3', 0.0, 'Infinity', None)
(4, 4, 'extra_4', 0.0, 'Infinity', None)

>>TABLE<< inventory
['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
(1, 1, 'inventory_1', 0.0, 'Infinity', None)
(2, 2, 'inventory_2', 0.0, 'Infinity', None)
(3, 3, 'inventory_3', 0.0, 'Infinity', None)
(4, 4, 'inventory_4', 0.0, 'Infinity', None)

>>TABLE<< cost
['id', 'state', 'colName', 'lowerBound', 'upperBound', 'primalValue']
(1, 1, 'cost_1', 0.0, 'Infinity', None)
(2, 2, 'cost_2', 0.0, 'Infinity', None)
(3, 3, 'cost_3', 0.0, 'Infinity', None)
(4, 4, 'cost_4', 0.0, 'Infinity', N

In [243]:
"""
Process CONSTRAINTS
"""

for key in db_tables.keys():
    if 'mosdex_json' not in db_tables[key]:
        continue        
    if "CONSTRAINT" == db_tables[key]['mosdex_json']['CLASS']:
        for statement in db_tables[key]['mosdex_json']['QUERY']:
            
            insert_array = db_tables[key]['mosdex_schema']['NAME']
            select_array = statement['SELECT']
            from_array = statement['FROM']
            
            insert_stmt = "INSERT INTO " + key + '(' + ",".join(insert_array) + ')'
            select_stmt = "SELECT " + ",".join(select_array)
            from_stmt = "FROM " + ",".join(from_array)
            
            stmt = insert_stmt + ' ' + select_stmt + " " + from_stmt
            if "JOIN" in statement:
                join_array = statement['JOIN']
                stmt = stmt + " JOIN " + " JOIN ".join(join_array)
            
            if "WHERE" in statement:
                where_array = statement['WHERE']
                stmt = stmt + " WHERE " + " ".join(where_array)
            # print(stmt)
            with Session(engine) as session, session.begin():
                session.execute(text(stmt))
                
        with Session(engine) as session:
            stmt = "select * from " + db_tables[key]['table_instance_name']
            rows = session.execute(text(stmt)).fetchall()
            print(f"\n>>TABLE<< {db_tables[key]['table_instance_name']}")
            print(f"{Base.metadata.tables[key].columns.keys()}")
            for row in rows:
                print(row)
    




>>TABLE<< ctInventory
['id', 'state', 'rowName', 'sense', 'RHS', 'dualValue']
(1, 1, 'ctInventory_1', '==', 10.0, None)
(2, 2, 'ctInventory_2', '==', 0.0, None)
(3, 3, 'ctInventory_3', '==', 0.0, None)
(4, 4, 'ctInventory_4', '==', 0.0, None)

>>TABLE<< ctCost
['id', 'state', 'rowName', 'sense', 'RHS', 'dualValue']
(1, 1, 'ctCost_1', '==', 0.0, None)
(2, 2, 'ctCost_2', '==', 0.0, None)
(3, 3, 'ctCost_3', '==', 0.0, None)
(4, 4, 'ctCost_4', '==', 0.0, None)


In [244]:
"""
Process MATRIX
"""

for key in db_tables.keys():
    if 'mosdex_json' not in db_tables[key]:
        continue        
    if "MATRIX" == db_tables[key]['mosdex_json']['CLASS']:
        for statement in db_tables[key]['mosdex_json']['QUERY']:
            insert_array = db_tables[key]['mosdex_schema']['NAME']
            select_array = statement['SELECT']
            from_array = statement['FROM']
            
            insert_stmt = "INSERT INTO " + key + '(' + ",".join(insert_array) + ')'
            select_stmt = "SELECT " + ",".join(select_array)
            from_stmt = "FROM " + ",".join(from_array)
            stmt = insert_stmt + ' ' + select_stmt + " " + from_stmt
            
            if "JOIN" in statement:
                join_array = statement['JOIN']
                join_stmt = " JOIN " + " JOIN ".join(join_array)
                stmt = stmt + join_stmt
                
            if "WHERE" in statement:
                where_array = statement['WHERE']
                stmt = stmt + " WHERE " + " ".join(where_array)
            
            # print(stmt)
            with Session(engine) as session, session.begin():
                session.execute(text(stmt))
        
        with Session(engine) as session:
            stmt = "select * from " + db_tables[key]['table_instance_name']
            rows = session.execute(text(stmt)).fetchall()
            print(f"\n>>TABLE<< {db_tables[key]['table_instance_name']}")
            print(f"{Base.metadata.tables[key].columns.keys()}")
            for row in rows:
                print(row)
        


>>TABLE<< ctInventoryMatrix
['id', 'state', 'rowName', 'regularName', 'regularCoefficient', 'extraName', 'extraCoefficient', 'inventoryName', 'inventoryCoefficient', 'laggedInventoryName', 'laggedInventoryCoefficient']
(1, 1, 'ctInventory_1', 'regular_1', 1.0, 'extra_1', 1.0, 'inventory_1', -1.0, None, None)
(2, 2, 'ctInventory_2', 'regular_2', 1.0, 'extra_2', 1.0, 'inventory_2', -1.0, 'inventory_1', 1.0)
(3, 3, 'ctInventory_3', 'regular_3', 1.0, 'extra_3', 1.0, 'inventory_3', -1.0, 'inventory_2', 1.0)
(4, 4, 'ctInventory_4', 'regular_4', 1.0, 'extra_4', 1.0, 'inventory_4', -1.0, 'inventory_3', 1.0)

>>TABLE<< ctCostMatrix
['id', 'state', 'rowName', 'regularName', 'regularCoefficient', 'extraName', 'extraCoefficient', 'inventoryName', 'inventoryCoefficient', 'costName', 'costCoefficient', 'laggedCostName', 'laggedCostCoefficient']
(1, 1, 'ctCost_1', 'regular_1', 400.0, 'extra_1', 450.0, 'inventory_1', 20.0, 'cost_1', -1.0, None, None)
(2, 2, 'ctCost_2', 'regular_2', 400.0, 'extra_2', 